# IHLT Lab 9: Coreferences

**Author:** *Zachary Parent ([zachary.parent](mailto:zachary.parent@estudiantat.upc.edu))*

### 2024-12-02

**Instructions:**

- Consider the first paragraph in Alice's Adventures in Wonderland, by Lewis Carroll:
  - Alice was beginning to get very tired of sitting by her sister on the bank, 
  and of having nothing to do: once or twice she had peeped into the book her 
  sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversations?'
- It can be downloaded from:
  - http://www.gutenberg.org/files/11/11-0.txt
- Apply the spaCy coreference solver to the previous paragraph.
- Show the coreference chains.
- What do you think about them? Justify your answer.